TODO:
- [ ] change sentence_num to length_percentage

In [5]:
import spacy
import networkx as nx
import numpy as np
import pandas as pd

nlp = spacy.load("en_core_web_md")

In [2]:
text_to_sum = "WASHINGTON - The Trump administration has ordered the military to start withdrawing roughly 7,000 troops from Afghanistan in the coming months, two defense officials said Thursday, an abrupt shift in the 17-year-old war there and a decision that stunned Afghan officials, who said they had not been briefed on the plans. President Trump made the decision to pull the troops - about half the number the United States has in Afghanistan now - at the same time he decided to pull American forces out of Syria, one official said. The announcement came hours after Jim Mattis, the secretary of defense, said that he would resign from his position at the end of February after disagreeing with the president over his approach to policy in the Middle East. The whirlwind of troop withdrawals and the resignation of Mr. Mattis leave a murky picture for what is next in the United States’ longest war, and they come as Afghanistan has been troubled by spasms of violence afflicting the capital, Kabul, and other important areas.  The United States has also been conducting talks with representatives of the Taliban, in what officials have described as discussions that could lead to formal talks to end the conflict. Senior Afghan officials and Western diplomats in Kabul woke up to the shock of the news on Friday morning, and many of them braced for chaos ahead.  Several Afghan officials, often in the loop on security planning and decision-making, said they had received no indication in recent days that the Americans would pull troops out.  The fear that Mr. Trump might take impulsive actions, however, often loomed in the background of discussions with the United States, they said. They saw the abrupt decision as a further sign that voices from the ground were lacking in the debate over the war and that with Mr. Mattis’s resignation, Afghanistan had lost one of the last influential voices in Washington who channeled the reality of the conflict into the White House’s deliberations. The president long campaigned on bringing troops home, but in 2017, at the request of Mr. Mattis, he begrudgingly pledged an additional 4,000 troops to the Afghan campaign to try to hasten an end to the conflict. Though Pentagon officials have said the influx of forces - coupled with a more aggressive air campaign - was helping the war effort, Afghan forces continued to take nearly unsustainable levels of casualties and lose ground to the Taliban. The renewed American effort in 2017 was the first step in ensuring Afghan forces could become more independent without a set timeline for a withdrawal.  But with plans to quickly reduce the number of American troops in the country, it is unclear if the Afghans can hold their own against an increasingly aggressive Taliban. Currently, American airstrikes are at levels not seen since the height of the war, when tens of thousands of American troops were spread throughout the country.  That air support, officials say, consists mostly of propping up Afghan troops while they try to hold territory from a resurgent Taliban."

In [3]:
# sentences = []
# for splitted_sentences in doc.sents:
#     sentences.append(splitted_sentences.text.strip())
# print(len(sentences))

In [8]:
row = 0
def summerize (text,sum_percentage):
    doc = nlp(text)

    global row
    
    row += 1
    print("row: ",row)

    sentences = []
    for splitted_sentences in doc.sents:
        sentences.append(splitted_sentences.text.strip())

    sum_length = round(len(sentences) * sum_percentage/100)
    if sum_length < 1:
        sum_length = 1
    lemmatized_sents = []
    
    words_count = 0
    for sentence in sentences:
        words = []
        for word in nlp(sentence):
            if not word.is_stop and not word.is_punct:
                words.append(word.lemma_)
        lemmatized_sents.append(" ".join(words))
        words_count += len(words)
    # print("words_count: ", words_count)

    # Calculate the similarity matrix
    similarity_matrix = np.zeros((len(lemmatized_sents), len(lemmatized_sents)))
    for i in range(len(lemmatized_sents)):
        for j in range(len(lemmatized_sents)):
            similarity_matrix[i, j] = nlp(lemmatized_sents[i]).similarity(nlp(lemmatized_sents[j]))

    graph = nx.from_numpy_array(similarity_matrix)

    # Calculate the PageRank scores
    scores = nx.pagerank(graph)


    top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:sum_length]
    summarized_text = [sentences[i] for i in top_sentence_indices]

    sum_words_count = 0
    for sentence in summarized_text:
        for word in nlp(sentence):
            if not word.is_stop and not word.is_punct:
                sum_words_count += 1
    # print("sum_words_count:\t", sum_words_count)
    reached_compression = round(100 - (sum_words_count * 100 / words_count), 2)
    summarized_text = " ".join(summarized_text)
    # print("reached compression:\t", reached_compression)
    
    
    return summarized_text

# Print the summary
# summary = summerize(text_to_sum, 50)
# print(summary)
# print(" ".join(summary))



words_count:  259
sum_words_count:	 165
reached compression:	 36.29
row:  1


In [10]:
df = pd.read_csv("new_sum_1000_articles.csv")

In [13]:
df.head()

,title,content,summarizedText,tf_idf_summary
0,Michael Flynn Was Paid to Represent Turkey’s I...,WASHINGTON — The candidate he was advising ...,WASHINGTON — The candidate he was advising ...,President Trump made the decision to pull the...
1,Hillary Clinton Crashes Tribeca Film Festival,NEW YORK (AP) — The premiere of a virtual r...,NEW YORK (AP) — The premiere of a virtual r...,President Trump made the decision to pull the...
2,Former PM David Cameron Resigns From Parliament,David Cameron announced he will resign from th...,David Cameron announced he will resign from th...,President Trump made the decision to pull the...
3,Gavin Newsom: Trump’s Election a ’Leap Backwar...,California Lt. Governor Gavin Newsom described...,Governor Gavin Newsom described the election o...,President Trump made the decision to pull the...
4,Trump to Louisiana Flood Victims: ’When You Hu...,"BATON ROUGE, Louisiana — In a heartfelt mom...","BATON ROUGE, Louisiana — In a heartfelt mom...",President Trump made the decision to pull the...


In [12]:
df['TextRank_summary'] = df['content'].apply(lambda x: summerize(x, 0.5))

words_count:  761
sum_words_count:	 23
reached compression:	 96.98
row:  2
words_count:  25
sum_words_count:	 20
reached compression:	 20.0
row:  3
words_count:  98
sum_words_count:	 31
reached compression:	 68.37
row:  4
words_count:  264


PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [ ]:
# def build_similarity_matrix(sentences,stop_words):
#   #create an empty similarity matrix
#     similarity_matrix = np.zeros((len(sentences),len(sentences)))
#     for idx1 in range(len(sentences)):
#         for idx2 in range(len(sentences)):
#         if idx1!=idx2:
#             similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],sentences[idx2],stop_words)
#     return similarity_matrix
#     # Finally the main function to call all the above function in the pipeline.

# def generate_summary(text,top_n):
#   nltk.download('stopwords')    
#   nltk.download('punkt')
#   stop_words = stopwords.words('english')    
#   summarize_text = []
#   # Step1: read text and tokenize    
#   sentences = read_article(text)
#   # Step2: generate similarity matrix            
#   sentence_similarity_matrix = build_similarity_matrix(sentences,stop_words)
#   # Step3: Rank sentences in similarity matrix
#    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
#    scores = nx.pagerank(sentence_similarity_graph)
#   # Step4: sort the rank and place top sentences
#   ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
  
#   # Step5: get the top n number of sentences based on rank
#   for i in range(top_n):
#     summarize_text.append(ranked_sentences[i][1])
#   # Step6 : output the summarized version
#   return " ".join(summarize_text),len(sentences)